In [ ]:
from pyspark.sql.functions import col, current_timestamp,regexp_replace,when

In [ ]:
loan_repay_schema="loan_id string,total_principle_received float,total_interest_received float,total_late_fee_received float,total_payment_received float,last_payment_amount float,last_payment_date string,next_payment_date string"

columns_to_check=["total_principle_received","total_interest_received","total_late_fee_received","total_payment_received","last_payment_amount"]

In [ ]:
srv_date = dt.datetime.now().strftime('%Y-%m-%d')
print(srv_date)
srv_lnrp_Path = f"/user/{user_name}/lendingClub/srv/2024-01-07/loan_repayments/"
print(bv_lnrp_Path)
lrp_csv_dest = f"/user/{user_name}/lendingClub/bv/csv/loan_repayments/"
lrp_parq_dest = f"/user/{user_name}/lendingClub/bv/parquet/loan_repayments"
print(lrp_csv_dest)
print(lrp_parq_dest)

In [ ]:
lrp_DF = spark.read \
                .format("csv") \
                .option("header",True) \
                .schema(loan_repay_schema) \
                .option("path", srv_lnrp_Path) \
                .load()

id_lrp_DF = lrp_DF.withColumn("ingest_date", current_timestamp())

drop_lrp_DF = id_lrp_DF.na.drop(subset=columns_to_check)

In [ ]:
tpr_DF = drop_lrp_DF \
        .withColumn("total_payment_received", \
                    when(
                        (col("total_payment_received") == 0.0) & (col("total_payment_received")!= 0.0) \
                        , col("total_principle_received")+col("total_interest_received")+col("total_late_fee_received")) \
                    .otherwise(col("total_payment_received")))

not_null_tpr = tpr_DF.filter(col("total_payment_received") !=0.0)

In [ ]:
lpd_DF = not_null_tpr.withColumn("last_payment_date", when(col("last_payment_date") == 0.0, None) \
                                                        .otherwise(col("last_payment_date"))
                                ) \
                      .withColumn("next_payment_date", when(col("next_payment_date") == 0.0, None) \
                                                        .otherwise(col("next_payment_date"))
                                )

csv_lrp_count = lpd_DF.count()
addLog("ingested loan_repayments file with " + str(csv_lrp_count) + " records in BV")

In [ ]:
lpd_DF.write \
        .format("csv") \
        .mode("overwrite") \
        .option("header",True) \
        .option("path", lrp_csv_dest) \
        .save()

lpd_DF.write \
        .format("csv") \
        .mode("overwrite") \
        .option("header",True) \
        .option("path", lrp_parq_dest) \
        .save()

In [ ]:
bv_lrp_stat = "Success"